In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.lax as lax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import DeterministicPolicy,Policy, Critic, ensemblize

import flax
import flax.linen as nn

from functools import partial



class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: TrainState
    target_critic: TrainState
    actor: TrainState
    config: dict = nonpytree_field()
    
    
        
    
    @jax.jit    
    def reset_critic_optimizer(agent):
    
        new_opt_state = agent.critic.tx.init(agent.critic.params)
        new_critic = agent.critic.replace(opt_state=new_opt_state)
        
        return agent.replace(critic=new_critic)
        
    @partial(jax.jit,static_argnames=('num_steps',))  
    def update_critic(agent, transitions: Batch,idxs:jnp.array,num_steps:int):
        
        
        
        def one_update(agent, batch: Batch):
                
            new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        
            def critic_loss_fn(critic_params):
                next_actions = agent.actor(batch['next_observations'])
                concat_actions = jax.numpy.vstack([batch["actions"],next_actions])
                concat_observations = jax.numpy.vstack([batch["observations"],batch["next_observations"]])
                
                
                # concat_q,updates = agent.critic(concat_observations, concat_actions,
                #                         training=True,params=critic_params,mutable=['batch_stats'])
                
                concat_q = agent.critic(concat_observations, concat_actions,
                                        training=True,params=critic_params)
                
                
                q,next_q = concat_q[:256],concat_q[256:]
             
                target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q
                target_q = jax.lax.stop_gradient(target_q)
                
                critic_loss = ((target_q-q)**2).mean()
                
                #return critic_loss, updates
                return critic_loss, {}
            
            new_critic,_ = agent.critic.apply_loss_fn(loss_fn=critic_loss_fn, has_aux=True)
            # new_critic, updates = agent.critic.apply_loss_fn(loss_fn=critic_loss_fn, has_aux=True)
            # params = new_critic.params
            # params["batch_state"]=updates["batch_state"]
            # new_critic = new_critic.replace(params=params)
            
            
            #return agent.replace(rng=new_rng, critic=new_critic, target_critic=new_critic)
            return agent.replace(rng=new_rng, critic=new_critic)
        
        
        get_batch = lambda transitions,idx : jax.tree_map(lambda x : x[idx],transitions)
        
        
        # for i in range(num_steps):
        #     agent = one_update(agent,get_batch(transitions,idxs[i]))
        agent = jax.lax.fori_loop(0, num_steps, 
                        lambda i, agent: one_update(agent,get_batch(transitions,idxs[i])),
                        agent)
        
        return agent,{}
       
    

        
    @jax.jit
    def update_actor(agent, batch: Batch):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        def actor_loss_fn(actor_params):
            
            actions = agent.actor(batch['observations'], params=actor_params)
            qs = agent.critic(batch['observations'], actions)
            #q = qs.mean(axis=0)
            q = qs
            actor_loss = (-q).mean()
            
            return actor_loss, {
                'actor_loss': actor_loss,
              
            }

        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=actor_loss_fn, has_aux=True)

        return agent.replace(rng=new_rng,actor=new_actor,), {**actor_info}
        


    @jax.jit
    def sample_actions(agent,observations: np.ndarray) -> jnp.ndarray:
        actions = agent.actor(observations)
       
        return actions
    
 

def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 tau: float = 0.005,
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = DeterministicPolicy((64,64), action_dim=action_dim,final_fc_init_scale=1.0)

        actor_params = actor_def.init(actor_key, observations)['params']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=actor_lr))
        
        
        #critic_def = ensemblize(Critic, num_qs=2,split_rngs={"dropout":True})(hidden_dims)
        critic_def = Critic(hidden_dims=hidden_dims)
        critic_params = critic_def.init(critic_key, observations, actions,training=False)['params']
        critic = TrainState.create(critic_def, critic_params, tx=optax.adam(learning_rate=critic_lr))
        target_critic = TrainState.create(critic_def, critic_params)        


        config = flax.core.FrozenDict(dict(
            discount=discount,
            target_update_rate=tau,    
        ))

        return SACAgent(rng, critic=critic, target_critic=target_critic, actor=actor, config=config)
        


In [2]:

from jaxrl_m.rollout import PolicyRollout,rollout_policy


def f(anc_agent,obs,actor):

    actions = anc_agent.actor(obs, params=actor)
    qs = anc_agent.critic(obs, actions,params=anc_agent.critic.params,training=False)
    #q = jnp.minimum(q1, q2)
    #q = qs.mean(axis=0)
    q = qs
    
    return q
    

@jax.jit
def estimate_return(anc_agent,anc_return,acq_rollout:PolicyRollout,):
    
    acq_obs = acq_rollout.observations
    acq_masks = acq_rollout.disc_masks
  
    acq_actor = acq_rollout.policy_params
    acq_return = acq_rollout.policy_return
    
    anc_actor = anc_agent.actor.params
    
    acq_q = f(anc_agent,acq_obs,acq_actor)
    anc_q = f(anc_agent,acq_obs,anc_actor)
    
    adv = ((acq_q - anc_q)*acq_masks).sum()/5
    acq_return_pred = anc_return + adv
  
    
    return acq_return_pred,acq_return


def evaluate_critic(anc_agent,anc_return,policy_rollouts):

    y_pred,y= [],[]
    for policy_rollout in policy_rollouts:
        
        acq_return_pred,acq_return = estimate_return(anc_agent,anc_return,policy_rollout)
        y_pred.append(acq_return_pred),y.append(acq_return)
        
    y_pred,y = np.array(y_pred),np.array(y)
    a2 = jnp.clip(((y-y_pred)**2),a_min=1e-4).sum()
    b2=((y-y.mean())**2).sum()
    R2 = 1-(a2/b2)  
    
    return R2


In [3]:
    import os
    from functools import partial
    import numpy as np
    import jax
    import tqdm
    import gymnasium as gym


    from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
    import wandb
    from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
    from jaxrl_m.dataset import ReplayBuffer
    from collections import deque


    env_name='Ant-v4'
    seed=np.random.choice(1000000)
    eval_episodes=10
    batch_size = 256
    max_steps = int(1e6)
    start_steps = 50000                   
    log_interval = 5000
    #eval_interval = 10000

    wandb_config = default_wandb_config()
    wandb_config.update({
        'project': 'd4rl_test',
        'group': 'sac_test',
        'name': 'sac_{env_name}',
    })


    env = EpisodeMonitor(gym.make(env_name))
    eval_env = EpisodeMonitor(gym.make(env_name))
    setup_wandb({"bonjour":1})

    example_transition = dict(
        observations=env.observation_space.sample(),
        actions=env.action_space.sample(),
        rewards=0.0,
        masks=1.0,
        next_observations=env.observation_space.sample(),
        discounts=1.0,
    )

    replay_buffer = ReplayBuffer.create(example_transition, size=int(1e6))
    actor_buffer = ReplayBuffer.create(example_transition, size=int(5e3))

    agent = create_learner(seed,
                    example_transition['observations'][None],
                    example_transition['actions'][None],
                    max_steps=max_steps,
                    #**FLAGS.config
                    )

    exploration_metrics = dict()
    obs,info = env.reset()    
    exploration_rng = jax.random.PRNGKey(0)
    i = 0
    unlogged_steps = 0
    policy_rollouts = deque([], maxlen=10)
    with tqdm.tqdm(total=max_steps) as pbar:
        
        while i < max_steps:
        
            replay_buffer,actor_buffer,policy_rollout,policy_return,num_steps = rollout_policy(agent,env,exploration_rng,
                    replay_buffer,actor_buffer,
                    warmup=(i < start_steps))
            policy_rollouts.append(policy_rollout)
            unlogged_steps += num_steps
            i+=num_steps
            pbar.update(num_steps)
            
                
            if replay_buffer.size > start_steps:
            
            
                transitions = replay_buffer.get_all()
                idxs = jax.random.choice(a=replay_buffer.size, shape=(5000,256), replace=True,key=jax.random.PRNGKey(0))
                agent = agent.reset_critic_optimizer()
                agent, critic_update_info = agent.update_critic(transitions,idxs,5000)
                R2 = evaluate_critic(agent,policy_rollouts[-1].policy_return,policy_rollouts)

                
                actor_batch = actor_buffer.get_all()      
                agent, actor_update_info = agent.update_actor(actor_batch)    
                
                
                update_info = {**critic_update_info, **actor_update_info, 'R2_validation': R2}
                
                
                if unlogged_steps > log_interval:
                    exploration_metrics = {f'exploration/disc_return': policy_return}
                    wandb.log(exploration_metrics, step=int(i),commit=False)
                    train_metrics = {f'training/{k}': v for k, v in update_info.items()}
                    wandb.log(train_metrics, step=int(i),commit=False)
                    #wandb.log(exploration_metrics, step=i)
                    policy_fn = agent.actor
                    eval_info = evaluate(policy_fn, eval_env, num_episodes=eval_episodes)
                    eval_metrics = {f'evaluation/{k}': v for k, v in eval_info.items()}
                    print('evaluating')
                    wandb.log(eval_metrics, step=int(i),commit=True)
                    unlogged_steps = 0


2023-12-19 02:56:01.602824: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


Extra kwargs: {'max_steps': 1000000}


  5%|▌         | 50414/1000000 [00:19<02:51, 5528.23it/s]

evaluating


  6%|▌         | 56449/1000000 [00:41<53:12, 295.54it/s]  

...

evaluating


 99%|█████████▉| 991255/1000000 [1:49:00<00:58, 149.51it/s]

evaluating


1000820it [1:51:02, 150.21it/s]                            

evaluating


In [4]:
def f(i,carry):
    return carry+i

a = np.array([1,2,3,4])

lax.fori_loop(0, 10, f,0)


Array(45, dtype=int32, weak_type=True)